In [ ]:
from matplotlib import pyplot as pyplot
from skimage.feature import hog
from skimage import exposure
from skimage.transform import resize

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import os
import numpy as np
import nibabel as nib



In [ ]:
def process_img(filepath):
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    return scan


In [ ]:
resize_l = 250
resize_w = 250


data_folder = f'{os.getcwd()}/labeled_data/data/'

length = 0
for label in os.listdir(data_folder):
    length += len(os.listdir(f'{data_folder}/{label}/'))


# X = np.empty(shape=(length, resize_l, resize_w, 64,), dtype=np.float16)
X = []
Y = np.empty(shape=(length), dtype=np.int64)

j = 0
print('Loading Images')
for label in os.listdir(data_folder):
    img_directory = f'{data_folder}/{label}/'
    for img in os.listdir(img_directory): 

        img_path = '{}{}'.format(img_directory, img)
        img = process_img(img_path)
        img = resize(img, output_shape=(resize_l, resize_w, 64))
        img = hog(img, orientations=8, pixels_per_cell=(20,20), cells_per_block=(1,1), channel_axis=-1)
        X += [img]
        Y[j] = label
        j += 1
        

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3)


In [ ]:
classifier = SVC()

classifier.fit(train_x, train_y)
y_pred = classifier.predict(test_x)

acc = np.mean(y_pred == test_y)
acc